In [2]:
# libs maths
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import sys

sys.path.append("../../")

# lib graficas
import cufflinks as cf
import plotly.plotly as py
import plotly.graph_objs as go
%matplotlib inline

%load_ext autoreload
%autoreload 2

from genesis.persistence.CollectionEODDataPersistence import CollectionEODDataPersistence

## OHLC DataStructure
Para cada objeto de dados e recebido um dataframe

In [3]:

collection = CollectionEODDataPersistence.get(exchange="NYSE", ticker="AA")
ohlc_df = collection.to_df()

ohlc_df.tail()


,exchange,ticker,date,open,high,low,close,volume
3444,NYSE,AA,2018-08-06,42.53,42.98,42.210,42.72,2936948.0
3445,NYSE,AA,2018-08-07,43.21,43.40,41.900,41.97,2883202.0
3446,NYSE,AA,2018-08-08,42.00,44.32,41.945,43.70,6483872.0
3447,NYSE,AA,2018-08-09,44.27,44.40,42.810,42.99,3052993.0
3448,NYSE,AA,2018-08-10,42.45,45.17,42.050,44.98,7593395.0


## Processo de Calculo dos indicadores

In [150]:
##indicator = MVH(df)
from genesis.calc.methods import *
from genesis.calc.common import *

def calc(dataframe):
    dataframe["volrel"] = VolRel(dataframe)
    dataframe["volfin"] = VolFin(dataframe)
    dataframe["indfor"] = IndFor(dataframe)
    dataframe["body_of_candle"] = BodyOfCandle(dataframe)
    dataframe["amp_rel"] = AmpRel(dataframe)
    #dataframe["rompLastTF"] = RompLastTF(dataframe)
    dataframe["trend"] = Trend(dataframe)
    
    dataframe["distmm_21"] = DistMM(dataframe)
    #dataframe["MovLastRomp"] = MovLastRomp(dataframe)
    #dataframe["sinceTF"] = SinceLastTF(dataframe)
    dataframe["romp_maxmin"] = RompMinMax(dataframe)
    dataframe["correction_bull"] = CorrectionBull(dataframe)
    dataframe["correction_bear"] = CorrectionBear(dataframe)
    
    dataframe["roc"] = ROC(dataframe, period=1)
    dataframe["mvh"] = MVH(dataframe)
    dataframe["vh"] = VH(dataframe)
    #dataframe["stddev"] = STDEV_jit(dataframe, 21)
    #dataframe["zigzag"] = ZIGZAG_LEANDRO(dataframe)
    return dataframe
indicators = calc(ohlc_df)

Executar os calculos

In [4]:
df_indicators = pd.read_csv("d:/indicators_NYSEA.csv")

,exchange_ticker,AmpRel,BBandBot,BBandTop,BodyOfCandle,DistMM,HIGH,IndFor,LOW,MVH,...,Trend_Precomedia1,VH,VolFin,VolRel1,VolRel2,ifr14,ifr2,ticker,date,update_at
0,NYSE_A_2018-08-10,0.841932,62.364976,68.124024,-0.595745,9.547992,66.87,2.653026,65.93,1.774619,...,1.0,19.644778,1.415013e+08,1.063255,1.090278,56.125691,20.246345,A,2018-08-10,2018-08-14 14:11:31.707691


In [151]:
indicators[indicators.romp_maxmin>5].tail()

,exchange,ticker,date,open,high,low,close,volume,volrel,volfin,...,body_of_candle,amp_rel,trend,distmm_21,romp_maxmin,correction_bull,correction_bear,roc,mvh,vh
3294,NYSE,AA,2017-12-28,52.05,54.20,52.021,54.14,5696354.0,1.421585,3.084006e+08,...,0.959156,1.449245,NaN,60.765053,6,54.14,5414.0,4.436728,3.493667,38.977810
3296,NYSE,AA,2018-01-02,54.06,55.22,53.910,55.17,2928905.0,0.741608,1.615877e+08,...,0.847328,0.865900,NaN,58.134622,6,55.17,5517.0,2.413217,3.469048,33.215000
3302,NYSE,AA,2018-01-10,54.37,56.26,54.230,56.17,5003205.0,1.226618,2.810300e+08,...,0.886700,1.274691,NaN,37.868420,6,56.17,5617.0,3.634686,3.404333,34.237058
3303,NYSE,AA,2018-01-11,56.60,57.13,55.870,56.91,2725442.0,0.678706,1.551049e+08,...,0.246032,0.783349,NaN,37.503579,6,56.91,5691.0,1.317429,3.411762,34.179690
3369,NYSE,AA,2018-04-18,58.26,60.28,57.250,59.40,8102930.0,1.963680,4.813140e+08,...,0.376238,1.245242,NaN,53.504310,6,59.40,5940.0,4.064471,3.813667,44.707986


## Processo de Calculo dos Filtros

Methodo Leandro

In [152]:
def filter_method_leandro(dataframe):
    dataframe["filter_method_leandro"] = dataframe.mvh >40
    return dataframe

    


filters_leandro = filter_method_leandro(indicators)


filters_leandro.tail(5)

,exchange,ticker,date,open,high,low,close,volume,volrel,volfin,...,amp_rel,trend,distmm_21,romp_maxmin,correction_bull,correction_bear,roc,mvh,vh,filter_method_leandro
3441,NYSE,AA,2018-07-31,42.68,43.61,42.6552,43.27,2521255.0,0.589291,1.090947e+08,...,0.653232,NaN,-9.927966,0,43.27,4327.0,1.620479,4.248619,56.579805,False
3442,NYSE,AA,2018-08-01,43.01,43.01,41.9000,42.04,3884133.0,0.890649,1.632890e+08,...,0.787410,NaN,-15.095890,0,42.04,4204.0,-2.842616,4.347095,56.922460,False
3443,NYSE,AA,2018-08-02,41.63,42.75,41.4600,42.48,2570123.0,0.580974,1.091788e+08,...,0.924062,NaN,-11.802293,0,42.48,4248.0,1.046622,4.449667,57.142861,False
3444,NYSE,AA,2018-08-03,42.54,43.30,42.3100,42.72,2101602.0,0.475826,8.978044e+07,...,0.700273,NaN,-9.441367,0,42.72,4272.0,0.564972,4.547571,56.418322,False
3445,NYSE,AA,2018-08-06,42.53,42.98,42.2100,42.72,2936948.0,0.659501,1.254664e+08,...,0.552543,NaN,-8.264167,0,42.72,4272.0,0.000000,4.646762,56.327808,False


Outro Exemplo

In [154]:
def filter_preco_acima_media(dataframe, indicators = [""]):
    dataframe["filter"] = dataframe.distmm_21 > 10
    return dataframe

filters_acima_media = filter_preco_acima_media(indicators)
filters_acima_media.tail(5)

,exchange,ticker,date,open,high,low,close,volume,volrel,volfin,...,trend,distmm_21,romp_maxmin,correction_bull,correction_bear,roc,mvh,vh,filter_method_leandro,filter
3441,NYSE,AA,2018-07-31,42.68,43.61,42.6552,43.27,2521255.0,0.589291,1.090947e+08,...,NaN,-9.927966,0,43.27,4327.0,1.620479,4.248619,56.579805,False,False
3442,NYSE,AA,2018-08-01,43.01,43.01,41.9000,42.04,3884133.0,0.890649,1.632890e+08,...,NaN,-15.095890,0,42.04,4204.0,-2.842616,4.347095,56.922460,False,False
3443,NYSE,AA,2018-08-02,41.63,42.75,41.4600,42.48,2570123.0,0.580974,1.091788e+08,...,NaN,-11.802293,0,42.48,4248.0,1.046622,4.449667,57.142861,False,False
3444,NYSE,AA,2018-08-03,42.54,43.30,42.3100,42.72,2101602.0,0.475826,8.978044e+07,...,NaN,-9.441367,0,42.72,4272.0,0.564972,4.547571,56.418322,False,False
3445,NYSE,AA,2018-08-06,42.53,42.98,42.2100,42.72,2936948.0,0.659501,1.254664e+08,...,NaN,-8.264167,0,42.72,4272.0,0.000000,4.646762,56.327808,False,False


## PRocesso de Sinal de Compra e Venda

In [156]:
def method_leandro(dataframe, filters = [""]):
    dataframe["buy"] = dataframe.mvh >40
    dataframe["sell"] = dataframe.mvh < 40
    return dataframe


df = method_leandro(filters_leandro)
df.tail()

,exchange,ticker,date,open,high,low,close,volume,volrel,volfin,...,romp_maxmin,correction_bull,correction_bear,roc,mvh,vh,filter_method_leandro,filter,buy,sell
3441,NYSE,AA,2018-07-31,42.68,43.61,42.6552,43.27,2521255.0,0.589291,1.090947e+08,...,0,43.27,4327.0,1.620479,4.248619,56.579805,False,False,False,True
3442,NYSE,AA,2018-08-01,43.01,43.01,41.9000,42.04,3884133.0,0.890649,1.632890e+08,...,0,42.04,4204.0,-2.842616,4.347095,56.922460,False,False,False,True
3443,NYSE,AA,2018-08-02,41.63,42.75,41.4600,42.48,2570123.0,0.580974,1.091788e+08,...,0,42.48,4248.0,1.046622,4.449667,57.142861,False,False,False,True
3444,NYSE,AA,2018-08-03,42.54,43.30,42.3100,42.72,2101602.0,0.475826,8.978044e+07,...,0,42.72,4272.0,0.564972,4.547571,56.418322,False,False,False,True
3445,NYSE,AA,2018-08-06,42.53,42.98,42.2100,42.72,2936948.0,0.659501,1.254664e+08,...,0,42.72,4272.0,0.000000,4.646762,56.327808,False,False,False,True


In [66]:
df_final_indicators = pd.read_csv("../data/indicators.csv", sep=";", decimal=",")
df_final_indicators.tail()

,Unnamed: 0,created_at,exchange,name,ticker,value
58203,58203,2018-08-07 09:45:14.456362,NYSEARCA,Corpo,XMPT,0.923846
58204,58204,2018-08-07 09:45:14.456362,NYSEARCA,AmplitudeRelativa,XMPT,1.477846
58205,58205,2018-08-07 09:45:14.458314,NYSEARCA,DistMM21,XMPT,-1.346154
58206,58206,2018-08-07 09:45:14.459288,NYSEARCA,MVH,XMPT,2.885498
58207,58207,2018-08-07 09:45:14.471980,NYSEARCA,Indicator_Romp,XMPT,NaN
